In [1]:
import numpy as np
import numpy.linalg.linalg as LA

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from matplotlib.transforms import Affine2D
import matplotlib.patches as patches
from matplotlib.text import Annotation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import random
from IPython.display import clear_output
import math

import scipy.stats as stats
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from scipy.spatial.distance import euclidean
from scipy.optimize import leastsq
from scipy.stats import multivariate_normal

twoPi = np.pi * 2

def find(condition):
    res, = np.nonzero(np.ravel(condition))
    return res

class MultipleNRV(object):
    """Multiple independent normal random variables."""
    def __init__(self, size, loc=0., scale=1.):
        self.size = size
        self.mean, self.std = loc, scale
        self.twoVariance = 2 * self.std ** 2
    
    def pdf(self, xs):
        """Returns the probability density function value for a particular
        vector."""
        twoVar = self.twoVariance
        if twoVar == 0:
            return 1 if xs == self.mean else 0
        else:
            delta2 = (xs - self.mean) ** 2
            return np.product( np.exp( -delta2 / twoVar ) / np.sqrt( twoVar * np.pi) )        
            
    def sample(self):
        """Returns a vector sampled from the PDF."""
        loc, scale, n = self.mean, self.std, self.size
        return loc if scale == 0 else np.random.normal(loc, scale, size=self.size)                                                                        

class World(object):
    
    def __init__(self, sensor_angles=(0,), luminance=1.0, light_coords=(10,0, -0.1), v_max=1.0, agent_radius=0.5, sensor_noise=0.12, motor_noise=0.25, dt=0.1, seed=None):

        self.sensors = np.array(sensor_angles)
        self.light_pos = np.array(light_coords)
        self.v_max = v_max
        self.agent_radius = agent_radius
        self.luminance = luminance
        self.dt = dt
        
        if seed is not None:
            np.random.seed(seed)

        # set up noise random variables
        sensor_sigma = sensor_noise * np.sqrt(dt)
        motor_sigma = motor_noise * np.sqrt(dt)
        self.sensor_rv = MultipleNRV(size=len(sensor_angles), scale = sensor_sigma)
        self.motor_rv = MultipleNRV(size=2, scale = motor_sigma)
                
    def sensor_pos(self, state):
        """Returns an array corresponding to a list of (x, y, 0) sensor 
        positions in world coordinates."""
        sensors, r = self.sensors, self.agent_radius
        x, y, theta = state
        n = len(sensors)
        
        result = np.zeros( (n, 3) )
        # copy robot x, y into sensors
        result[:,0:2] = state[0:2]
        
        angles = theta + sensors
        result[:,0] = r * np.cos(angles) + x
        result[:,1] = r * np.sin(angles) + y
        
        return result
        
    def sensor_input(self, state):
        """Returns an array of raw sensor input values for a particular
        agent state (position and orientation). These are calculated
        according to an inverse square distance law, and the agent's body
        can occlude a sensor reducing its input to zero.
        """        
        # various relevant parameters
        r, K = self.agent_radius, self.luminance
        # light position
        l_pos = self.light_pos        

        # unpack 3D position and heading from (x, y, theta) state
        pos, theta = np.array(tuple(state[0:2]) + (0,)), state[-1]        

        # positions in world coordinates of each sensor
        s_pos = self.sensor_pos(state)
        # array of distances of sensors from light source
        d_s = LA.norm(l_pos - s_pos, axis=1)
        
        # distance of light from robot's centre
        d_0 = LA.norm(l_pos - pos)

        # array of zeros or ones for each sensor according to whether the 
        # agent's body lies between the sensor and the light source
        not_occluded = (d_0**2 >= r**2 >= (d_s**2 - d_0**2))
        
        # light reaching each sensor
        return not_occluded * K / d_s **2                
        
    def sensor_transform(self, activation):
        """Returns a vector of sensor readings for a particular sensor input 
        value (activation) vector. Noise is usually applied to the activation 
        before applying the transform."""
        # rescale to (0, 1) interval, assuming activation is positive
        #return activation / (1 + activation)
        K, l_pos = self.luminance, self.light_pos
        # minimum distance is z coordinate of the light position
        d_min = l_pos[-1]
        
        # rescale activation to range between 0 and a_max
        # with midpoint around 
        a_max = K / ( d_min ** 2 )
        a = a_max / (1 + np.exp(5*(K/4 - activation)))
        
        #return 1 / (1 + np.exp(-activation))
        return activation
        #return np.sqrt(K / a)
    
    def sensor_inverse_transform(self, reading):
        """Returns the vector of sensor input values (activations) that would be 
        needed to produce the specified sensor reading. """
        return reading / (1 - reading)
    
    
    def sense(self, state):
        """Returns a vector of sensor reading values for a 
        particular agent state (position and orientation). 
        Noise is added to the raw luminance at the sensor's location
        and the result is rescaled to the (0, 1) interval.
        """
        activation = self.sensor_input(state) + self.sensor_rv.sample()
                        
        # and rescale to (0, 1) interval
        return self.sensor_transform(activation)
    
    def p_sensation(self, state, sensation):
        """Returns a probability density value for the likelihood of a 
        particular sensor reading vector given a particular agent state."""
        # invert rescaling operation to find the original activations 
        sensor_activation = self.sensor_inverse_transform(sensation)
        # determine the actual luminance at the sensors
        sensor_input = self.sensor_input(state)        
        
        # interrogate the RV object to get the PDF value
        return self.sensor_rv.pdf(sensor_input - sensor_activation)        

    def act(self, state, action):
        """Applies a motor activation vector to an agent state, and simulates 
        the consequences using Euler integration over a dt interval."""
        # noisily map the action values to a (-1, +1) interval
        motor_out = self.v_max * np.tanh(action) + self.motor_rv.sample()
        
        # calculate the linear speed and angular speed
        v = motor_out.mean()
        omega = (motor_out[1] - motor_out[0]) / (2.0 * self.agent_radius)
        
        # calculate time derivative of state
        theta = state[-1]
        deriv = [ v * np.cos(theta), v * np.sin(theta), omega ]
        
        # perform Euler integration
        return self.dt * np.array(deriv) + state 
    
    
    # Function for changing the light position, here we can take the light, move it for N timestamps (in this case 100 or 10 seconds) and move it back
    def light_change(self,original_state,active_token,stamp,light_poses):
        
        if active_token == 2: # If the token given is 2 initially we won't get a light change
            pass
        
        if active_token == 0: # change light from its intiailised location to a random point along the x=10 axis, as it was found, too far away from the light,t he robot will nto get distracted
            original_state = self.light_pos # initialisied light location for replacement
            light_coords = [10,random.uniform(-5,5),-0.1]
            self.light_pos = np.array(light_coords) # officially change light position in simulation
            new_active_token = 1 #change token for next round
            light_poses.append(light_coords) # add new coordinates to light locations

            
        if active_token == 1: # if the light had been moved here we move the light back to its initialised location
            self.light_pos = original_state # officially change light location in code
            new_active_token = 0 # change our token back
            self.luminance = 1 

        stamp = stamp + 100
            
        return(original_state,new_active_token,stamp,light_poses,self.light_pos)        
        
    def simulate(self, controller,active_token, interval=500.0):
        """Simulates the agent-environment system for the specified interval
        (in simulated time units) starting from a random state. Returns
        a (poses, sensations, actions, states) tuple where poses is a time array 
        of agent poses (position and orientation), sensations is a time array of 
        sensory readings, actions is a time array of motor activations, and
        states is a list of arbitrary internal controller state objects.
        
        Must be called with a controller function of the form 
        controller(sensation, state, dt) that returns a (action, state) tuple
        outputting motor activations and updated internal state in
        response to sensor readings.
        """
        poses = [ self.random_state() ]

        states = [ None ]
        sensations = [ ]
        actions = [ ]
        
        motor_history =   []         # Robot keeps track of its own motor positions
        local_angle_history  =   [0] # Initialise local angle history
        global_angle_history =   [0] # Initialise global angle history 
                                     # (all of this is relative to the journey of the robot and thus the starrt angles arent important)
        slope_l = []                 # List of graients found by the light to attempt to tell if the robot has found the most
                                     # light intensity for a given angle
        

        self_location   = [[0,0]] # Here we need to define where the robot thinks it starts, 
                                  # it doesnt matter if it actually starts at say [1,1] 
                                  #all that matters is where it thinks its location RELATIVE to its start position
        target_angle    = 0
        sense_list      = []    
        
        reached = 0               # Tokens that the robot can change if it deems it has found the light or home
        reached_home = 0

        
        original_state = self.light_pos # make sure that the intialised location can be founda gain by the light movement function
        
        light_poses = [] # if we choose to change the light location, this keeps track of teh new location for plotting
        
        stamp = 0 # this gives us a timestamp that the light can be in it's temporary location for before reverting back to its original
                  # position
        
        guess_angle = 0 # for the bayesian calculation, we need a guess of how far and at what angle the light is reletive to the robot, this variable is for the controller to process and pass
        
        for i in range(int( interval / self.dt )):
            
            if i%1000 == 0 and active_token == 0: # here every 100 seconds the light will move 
                original_state,active_token,stamp,light_poses,self.light_pos = self.light_change(original_state,active_token,i,light_poses)
                
            if stamp == i  and active_token == 1: # here we call and move the light back when the allotted decoy time is up
                original_state,active_token,stamp,light_poses,self.light_pos = self.light_change(original_state,active_token,i,light_poses)

            sensations.append(self.sense(poses[-1]))
            action, local_angle_history,self_location,global_angle_history,reached,reached_home,slope_l,guess_angle,state = controller(sensations, states[-1], self.dt,motor_history,local_angle_history,self_location,target_angle,global_angle_history,reached,reached_home,slope_l,guess_angle)     
            actions.append(action)
            states.append(state)
            poses.append(self.act(poses[-1], actions[-1]))

        return np.array(poses), np.array(sensations), np.array(actions), states,self.light_pos,light_poses,self_location,guess_angle 
        
    def random_state(self):
        """Returns a random initial state."""
        result = np.zeros(3)
        result[-1] = np.random.rand() * twoPi
        
        return result
        
        
    def task1fitness(self, poses):
        """Returns the fitness of the trajectory described by poses on 
        assignment task 1 (reaching the light source)."""
        return -self.reached_light_at(poses)
        
    def task2fitness(self, poses):
        """Returns the fitness of the trajectory described by poses on 
        assignment task 1 (reaching the light source and returning to base)."""
        light_time = self.reached_light_at(poses)
        if light_time == np.inf:
            return -np.inf
        return -self.first_reached(poses, np.array([0, 0]), after=light_time) 
        
    def first_reached(self, poses, xy, after = 0, within = 1.5):
        after_index = int(np.floor(after / self.dt))
        ds = LA.norm(xy - poses[after_index:,0:2], axis=1)
        indices = np.nonzero(ds < within)[0]
        
        if len(indices) == 0:
            return np.inf
        
        return (indices[0] + after_index) * self.dt
        
    def reached_light_at(self, poses):
        return self.first_reached(poses, self.light_pos[0:2])
    
    
    def animate(self, poses, sensations, speedup=5):
        r, l_pos = self.agent_radius, self.light_pos
        x, y, theta = poses[0]
        
        # use an Ellipse to visually represent the agent's body
        body = patches.Ellipse(xy=(0, 0), width=2 * r, height=2 * r, fc='w', ec='k')
        # use a black dot to visually represent each sensor
        sensors = [ patches.Ellipse(xy=(r * np.cos(theta), r * np.sin(theta)), width=0.2, height=0.2, fc='b') for theta in self.sensors ]
        # use small rectangles to visually represent the motors
        motors = [ patches.Rectangle((-0.5*r, y), width = r, height = 0.2*r, color="black") for y in (-1.1*r, 0.9*r) ]
        # use a line to indicate the agent's orientation
        line = Line2D( (x, x + r * np.cos(theta)), (y, y + r * np.sin(theta)) )
        line = Line2D( (0, r), (0, 0) )
        # draw a line showing the agent's "trail"
        trail = Line2D( [], [], color='r') 
        # display a clock
        clock = Annotation('', (0.8, 0.9), xycoords='axes fraction')    
        # use a yellow circle to visually represent the light
        light_r = patches.Ellipse(xy=l_pos[0:2], width=1, height=1, fc='y', ec='none')
        light = patches.Ellipse(xy=l_pos[0:2], width=0.25, height=0.25, fc='b')
        
        fig, (ax1, ax2) = plt.subplots(2, 1, gridspec_kw = {'height_ratios': [10, 1] } )
        ax1.axis("equal")
        ax1.set_xlim([-15, 15])
        ax1.set_ylim([-15, 15])
        ax1.set_title("Click On Main Display To Pause / Unpause")
        
        ax2.set_title("Click On Sensor Graph To Change Time")
        
        tracker = ax2.axvline(0, 0, 1)
        paused = [ False ]
        last_index = [ -1 ]
        t_index = [ 0 ]
        
        if sensations is not None:
            times = np.arange(0, self.dt * len(sensations), self.dt)
            # plot the sensor values
            ax2.plot(times, sensations, 'k');
            # plot the ideal (noiseless) sensor values
            ideal = np.array([self.sensor_transform(self.sensor_input(pose)) for pose in poses[:-1]])
            print(ideal[0])
            ax2.plot(times, ideal, 'r')
    
        def draw(index):
            if not paused[0]:
                t_index[0] = t_index[0] + (index - last_index[0])
                t_index[0] = t_index[0] % len(poses)
    
            last_index[0] = index
                
            x, y, theta = poses[t_index[0]]
            tr = Affine2D().rotate(theta).translate(x, y) + ax1.transData
            
            agent_patches = (body, line) + tuple(sensors) + tuple(motors)
            
            for patch in agent_patches:
                patch.set_transform(tr);
                
            trail.set_data( poses[:t_index[0], 0], poses[:t_index[0], 1] )
            
            time = t_index[0] * self.dt
            tracker.set_xdata([time, time])
                
            clock.set_text("Time: %.02f" % time)
                    
            return (trail, light_r, light, clock, tracker) + agent_patches
        
        def init():
            result = draw(0)
            for artist in result:
                if artist is not tracker:
                    ax1.add_artist(artist)
            return result
    
        def onclick(event):
            if event.button == 1:
                # pause if the user clicks on the main figure
                if event.inaxes is ax1:
                    paused[0] = not paused[0]
                # edit time directly if the user clicks on the graph over time
                elif event.inaxes is ax2:
                    t_index[0] = (int) (event.xdata / self.dt)            
            
        def anim(index):
            return draw(index)
        
        
        ani = FuncAnimation(fig, anim, init_func=init, frames = None, interval=1000*self.dt/speedup, blit=True, save_count=len(poses))
        
        plt.show()
    
        fig.canvas.mpl_connect('button_press_event', onclick)
 
        return ani            
                                  

In [2]:
# This function defines the circulatiry of a set of coordinates, the more circular a path, the more aimless
# a search strategy appeasr to be, thus circularity is a good measure of purpose (or lack thereof)

def circul(x,y,graph):

    def x_y_tuple(x,y):
        list_coords_tuple = []
        for i in range(len(x)):
            list_coords_tuple.append([x[i], y[i]])
        return(list_coords_tuple)

    def perim(points,graph):
        hull = ConvexHull(points)
        if graph ==1 :
            plt.plot(points[:,0], points[:,1], 'o')
            for simplex in hull.simplices:
                plt.plot(points[simplex, 0], points[simplex, 1], 'r')
        perimeter = hull.area
        return(perimeter)
    
    def area_cal(pts):
        hull = ConvexHull(pts) 
        area = hull.volume
        return(area)
    
    pts = np.array(x_y_tuple(x,y))
    perimeter = perim(pts,graph)
    area = area_cal(pts)

    circularity = 4*math.pi*area/((perimeter**2))
    
    return(area, perimeter, circularity)



In [3]:
# Here take two guassian curves and find their intersection distance (this is applied to the shortest line between the 2 3d curves)
def solve(m1,m2,std1,std2):
    a = 1/(2*std1**2) - 1/(2*std2**2)
    b = m2/(std2**2) - m1/(std1**2)
    c = m1**2 /(2*std1**2) - m2**2 / (2*std2**2) - np.log(std2/std1)
    
    return np.roots([a,b,c])

# here we are taking the  guessed light position, the guessed dead reckoning location and using the sensor and motor noise as
# staandard deviation, so the larger the noise the higher the uncertainty, the wider the guassian 'hill'

def gaussian_intersection(light,motor,sensor_noise,motor_noise):
    
    # we find the shortest distance between the two guassian peaks with basic triganometry
    
    dist = np.sqrt((motor[0]-light[0])**2 + (motor[1]-light[1])**2) 
    
    # send the distance through to find where the intesction point will be along this line
    
    mid = solve(0,dist,sensor_noise,motor_noise)
    mid = [item for item in mid if item >= 0]
    
    # if lines overlap on both sides of a guasian peak this won't help, we only want to take the mid point between the guassian peaks
    
    mid = min(mid)
    
    # Figure out the mid point on the 2D plane

    if motor[0] == light[0] and motor[1]>light[1]:
        loc = [motor[0],light[1]+mid]
    
    if motor[0] == light[0] and light[1]>motor[1]:
        loc = [motor[0],motor[1]+mid]
        
    if motor[1] == light[1] and motor[0]>light[0]:
        loc = [light[0]+mid,motor[1]]
    
    if motor[1] == light[1] and light[0]>motor[0]:
        loc = [motor[0]+mid,motor[1]]
    
    if motor[0] == light[0] and motor[1] == light[1]:
        loc = [motor[0],light[0]]
    
    if motor[0] != light[0]:
        x_dif = abs(motor[0]-light[0])
        y_dif = abs(motor[1]-light[1])
        
        angle = np.arctan(y_dif/x_dif)
    
    if motor[0]>light[0] and motor[1]>light[1]:
        loc = [light[0]+ mid*np.cos(angle),light[1]+ mid*np.sin(angle)]
        
    if motor[0]>light[0] and motor[1]<light[1]:
        loc = [light[0]+ mid*np.cos(angle),light[1]- mid*np.sin(angle)]
        
    if motor[0]<light[0] and motor[1]<light[1]:
        loc = [light[0]- mid*np.cos(angle),light[1]- mid*np.sin(angle)]
        
    if motor[0]<light[0] and motor[1]>light[1]:
        loc = [light[0]- mid*np.cos(angle),light[1] + mid*np.sin(angle)] 
    
    # Return these mid point coordinates as the new location

    return(loc)

In [4]:
# here we are taking the sense data for the robot's 'spin in place' behaviour to treat the light like a cephid star
# obiously we are moving and the lights luminosity is not changing, however we can use the oscillating light 
# as a guage of the light intensity minus noise, it isnt perfect but its fairly robust

def cephid_star(sensations,sampling_size,theta,lum=1,r=0.5,height=-0.1):
    
    sensations = np.array(sensations)
    t = np.linspace(0, sampling_size, sampling_size)

    data = sensations[sampling_size:2*sampling_size - len(sensations)]
    data = data.reshape(sampling_size,)

    guess_mean = np.mean(data)
    guess_std = np.std(data)/(2**0.5)/(2**0.5)
    guess_phase = 0
    guess_freq = 1
    guess_amp = 0.5

    data_first_guess = guess_std*np.sin(t+guess_phase) + guess_mean
    optimize_func = lambda x: x[0]*np.sin(x[1]*t+x[2]) + x[3] - data
    est_amp, est_freq, est_phase, est_mean = leastsq(optimize_func, [guess_amp, guess_freq, guess_phase, guess_mean])[0]
    data_fit = est_amp*np.sin(est_freq*t+est_phase) + est_mean

    fine_t = np.arange(0,max(t),0.1)
    data_fit=est_amp*np.sin(est_freq*fine_t+est_phase)+est_mean
    
    max_light_reading = abs(max(data_fit))

    distance = np.sqrt((lum/max_light_reading) - height**2 ) + r
    
    # Invert the light intesnsity and use the guessed angle provided by the controller at this stage to say where the light is

    # Convert information from polar to cartesian coordinates 
    
    lx = distance*np.cos(theta)
    ly = distance*np.sin(theta)

    return([lx,ly])


In [5]:
def BTDR_Bayes(sensations_list, state, dt,motor_history,local_angle_history,self_location,target_angle,global_angle_history,reached,reached_home,slope_l,guess_angle):
    
    #here we can pick the sampling rate, for one full rotation it is about 31-31 timesteps so we will be safe and say 31
    
    sampling_size = 31
    
    if len(sensations_list) <= sampling_size: # so to get an idea of the sensor noise (standard deviation for the gaussian curve)
                                              # lets sit in place for 31 timesteps and see what sensor variation we get
        output = [0,0] 
        slope = 0 # we are going to say that the slope information is 0 because wee need a minimum of 31 sample size of our sense
                  # data to get a graient with the code implemented, besides as this is noise determining behaviour, slope isnt important right now
                
    if len(sensations_list)>sampling_size: # okay now we have obtained sensory noise lets calculate the slope
        
        sensations_snip = (np.array(sensations_list[-sampling_size:])).reshape(sampling_size,)
    
        slope, intercept, r_value, p_value, std_err = stats.linregress(np.linspace(0,sampling_size,sampling_size),sensations_snip)
        
        # Here, the slope or gradient is used to integrate proior information to create a more informed threshold value for our 'Run' behaviour
        
        sensor_error = 3*(np.std(sensations_list[:sampling_size]))
        
        # Because motor noise and moving around will constantly change sensor readings we will keep the sensor error based on those first 31 points
        # testing it out, it was found the standard deviation multiplied by 3 is roughly equivalent for sensory values
        
        output = [1,-1] # lets 'Tumble' and look for the angle ascosiated with the sharpest sensory gradient

        counter = 0 # set a counter for our sampling
        sense_max = -100 # set a sensory reading lower than possible 
        index_max = 0 # token that keeps track of highest gradient timestep in past 31
        
        for i in range(sampling_size-1): # 'Tumble'for one full rotation

            if slope_l[-sampling_size:][i] >= sense_max: # If our gradient is high enough this will become the new timestamp to change our angle
                sense_max = slope_l[-sampling_size:][i] # change the maximum gradient value to comapre against 
                
                index_max = counter # counter to hold onto the local timestamp

            counter+=1 # move onto next timestamp
         
        target_angle = local_angle_history[-sampling_size:][index_max] # of the past full rotation is this the highest gradient we found if so this becaomse the new target angle to rotate to
        
        if len(sensations_list) == sampling_size*2: # for our first rotation in place we note 
                                                    # down the angle that had the steepest graient for our photsensor based guess of where the light is
            angle_guess = target_angle

    angle_change = ((np.tanh(output[1])-np.tanh(output[0]))/(2*0.5)) # We update a local angle set as if we would keep spinning indefinitely, this allows us to make the choise to wither go forward or spin
    new_angle = local_angle_history[-1] + angle_change*dt            # update the local angle history
    
   
    if target_angle+0.75 >new_angle >target_angle-0.75 and (sampling_size*2)>len(sensations_list)>=sampling_size:
        output = [1,-1] # This is for out cephid star stage, seeing what one cycle of spinning outputs as the light 
                        # intesnity of the light source
            
    if target_angle+0.75 >new_angle >target_angle-0.75 and len(sensations_list)>2*sampling_size:
        output = [1,1] # if the theoretical next (local) angle matches the target angle derived above we move toward the light source

    if reached == 1: # if the light source is percieved as reached we change our behaviour 
                
        net_angle = ((global_angle_history[-1])%(2*math.pi)) # here we are finding the angle of orientation of the robot (where its front is facing)
        home_angle = abs(np.arctan(self_location[-1][1]/self_location[-1][0])) # here we are finding the angle of the centre of the robot chissis is relative to its percieved start points
        
        if self_location[-1][1] >= 0 and  self_location[-1][0] > 0: # decide what angle we need to turn to to be facing home
            targ = math.pi + home_angle
            
        if self_location[-1][1] < 0 and  self_location[-1][0] < 0:
            targ = home_angle
        
        if self_location[-1][1] > 0 and  self_location[-1][0] < 0:
            targ = 2*math.pi - home_angle
            
        if self_location[-1][1] < 0 and  self_location[-1][0] > 0:
            targ = math.pi - home_angle
        
        if targ*1.1 > net_angle > targ*0.9:
            output = [1,1] # lets head home
                               
        if targ*1.1 < net_angle or net_angle < targ*0.9:
            output = [0,1] # let's turn around and face home based on above information
        
        if 0.5>output[0]>-0.5 and 0.5>output[1]>-0.5:
            reached_home = 1 # if we percieve ourr cordinates are approximately zero lets change the 'reached home token'

            
    if reached_home == 1:
        # if we have decided we have reached home lets shut off motors (this never happens in practice)

        output =[0,0]
        
    
    delta_angle = ((np.tanh(output[1])-np.tanh(output[0]))/(2*0.5)) # this notes the actual change in angle (instead of th etheoretical perpetual rotation we note what out motor outputs did)

    real_new_angle = global_angle_history[-1] + delta_angle*dt # euler integrate our new angle
    
    hyp = (((np.tanh(output[1])+np.tanh(output[0]))/2))  # find out our journey linkage (how far we moved in this time step)

    s_location = [(hyp*(np.cos(real_new_angle))*dt + self_location[-1][0]),(hyp*(np.sin(real_new_angle))*dt + self_location[-1][1])]
    self_location.append(s_location) # lets integrate our knowledge of angle and journey linkage to update our idea of self location

    local_angle_history.append(new_angle)
    global_angle_history.append(real_new_angle) # update our angle history
    
    slope_l.append(slope) # update our list of gradient

    if len(sensations_list) > sampling_size and slope>0.025 and reached == 0: # if our slope hits a certain threshold (this was trial and errored to find an appropriate value)
        
        # Okay, here we need to use senor and motor noise with percieved light and self location to update our knowledge of where we are for the return journey
        
        sensor_noise = sensor_error
        motor_noise =0.1
        guess_location = s_location
        guess_light = cephid_star(sensations_list,sampling_size,guess_angle)
        updated_location = gaussian_intersection(guess_light,guess_location,sensor_noise,motor_noise)
        self_location.append(updated_location) # Add our bayesian inspired new location to act on for the return journey

        reached = 1 # change our light reached token to reflect we think we have reached the light

    return(output,local_angle_history,self_location,global_angle_history,reached,reached_home,slope_l,guess_angle,None)


In [6]:
# Do a trial run to see if everything works

w = World(sensor_noise = 0.1,motor_noise=0.1)

act_token = 0

poses, sensations, actions, states,light_pos,light_poses,self_location,f_command = w.simulate(BTDR_Bayes,active_token=act_token)
self_location=np.array(self_location)
plt.ion()
%matplotlib qt

# Lets plot our outward and return jounrney ant style with our path integrated route home

if np.isinf(w.task1fitness(poses)) ==True:
    plt.plot(poses[:,0],poses[:,1],label = 'Outward Journey')
    
else:
    plt.plot(poses[:int(-w.task1fitness(poses)*10)][:,0], poses[:int(-w.task1fitness(poses)*10)][:,1],c='b',label = 'Outward Journey')
    plt.plot(poses[int(-w.task1fitness(poses)*10):][:,0], poses[int(-w.task1fitness(poses)*10):][:,1],c='r',label = 'Return Journey')

if len(light_poses)>0:
    plt.scatter((np.array(light_poses))[:,0],(np.array(light_poses))[:,1],c='y',label='decoy light')
plt.scatter(light_pos[0], light_pos[1],c='r',s=100,label = 'Final_light')

circ_num =circul(poses[:,0],poses[:,1],0)

plt.title('Bacterial tumbling and dead reckoning (bayesian) \n , path of robot \n Circularity = '+str(round(circ_num[2],3)))
plt.legend()


if act_token == 0:
    str1 = 'Bacterial tumbling and dead reckoning  (bayesian)controller with decoys, path of robot'
    
if act_token == 2:
    str1 = 'Bacterial tumbling and dead reckoning (bayesian)controller without decoys, path of robot'
    
if np.isinf(w.task1fitness(poses)) == False and np.isinf(w.task2fitness(poses)) == False:
    str2 = 'success'
    
else:
    str2 = 'Failure'
plt.savefig(str1+str2)

ani = w.animate(poses, sensations)

print("Fitness on task 1: %f" % w.task1fitness(poses))
print("Fitness on task 2: %f" % w.task2fitness(poses))
plt.show()

[0.01057698]
Fitness on task 1: -66.600000
Fitness on task 2: -inf


In [ ]:
# Here we can run the simulation many times to get a pcolor plot of successes with different levels of noise with 
# or without decoy lights

def test_func(act_token):
    
    res = 20
    
    sens_vars = np.linspace(0.1,1,res)
    mot_vars = np.linspace(0.1,1,res)
    
    results_outward =np.zeros((res,res))
    results_return =np.zeros((res,res))

    k = 0
    for s in sens_vars:
        l = 0 
        for m in mot_vars:
            w = World(sensor_noise =s,motor_noise=m)
            
            avg_o = 0
            avg_r = 0
            
            for i in range(5):
                
                poses, sensations, actions,light_pos,light_poses,light_poses,self_location,f_command = w.simulate(BTDR_Bayes,active_token=act_token)
                fit_1 = w.task1fitness(poses)
                fit_2 = w.task2fitness(poses)
    
                if np.isinf(fit_1)==True:
                    fit_1 = -1000
                
                if np.isinf(fit_2)==True:
                    fit_2 = -1000
                    
                avg_r+=fit_2
                avg_o+=fit_1
                
            results_outward[k][l] = avg_o/5
            results_return[k][l] = avg_r/5
            
            l+=1
            
        k+=1 
        clear_output()
        print(k*100/(res),'% Complete')
        

    
    fig, (ax0,ax1) = plt.subplots(1,2)

    c = ax0.pcolor(sens_vars,mot_vars,results_outward, cmap='RdBu')
    ax0.set_title('Outward journey successes')
    ax0.set_xlabel('Motor Noise')
    ax0.set_ylabel('Sensor Noise')
    ax0.axis([0.1, 1, 0.1, 1])
    fig.colorbar(c , ax=ax0)
            
    c = ax1.pcolor(sens_vars,mot_vars,results_return, cmap='RdBu')
    ax1.set_title('Return journey successes')
    ax1.set_xlabel('Motor Noise')
    ax1.set_ylabel('Sensor Noise')
    ax1.axis([0.1, 1, 0.1, 1])
    fig.colorbar(c , ax=ax1)
    
    if act_token == 0 :
        fig.suptitle('Bacterial tumbling with dead reckoning (bayesian) \n performance (w/ Decoy light)')
        fig.savefig('Bacterialtumblingwithdeadreckoningbayesianperformance(wDecoylight).PNG')
    if act_token == 2 :
        fig.suptitle('Bacterial tumbling with dead reckoning (bayesian) \n performance (No decoy light)')
        fig.savefig('Bacterialtumblingwithdeadreckoningbayesianperformance(NoDecoylight).PNG')

test_func(0)            